O treinamento do modelo se deu sobre um subconjunto dos dados que foi anotado manualmente.  
O conjunto anotado pode ser encontrado em `sample.csv`.

A amostragem foi feita através do script `sample.py`.

In [78]:
import numpy as np
import pandas as pd
import re

import seaborn as sns
from matplotlib import pyplot as plt

%matplotlib inline

In [79]:
annotated_data = pd.read_csv('sample.csv')
annotated_data.head()

,TITLE,smartphone
0,Capa Samsung Galaxy S5 Mini Pc Couro Branco,0
1,Cabo Usb Retrátil 3 Adaptadores Preto - Muvit,0
2,Celular Samsung Galaxy A8 Plus 2018 Sm-a730f 6...,1
3,Lousa Magnética Grande Em Formato De Vaca Suelen,0
4,Aplique Madeira E Papel Placa Pinguim Com Colh...,0


# Extração de features

A extração de *features* foi feita através de expressões regulares:

Se o título de uma instância der match em uma regex em qualquer ponto, associa-se o valor 1 àquele atributo.  
Caso contrário, é atribuído 0.

In [80]:
patterns_re = [
    re.compile(r'smart', re.IGNORECASE), # contém "smart"
    re.compile(r'(?:ph|f)one', re.IGNORECASE), # contém "fone" ou "phone"
    re.compile(r'celular', re.IGNORECASE), # contém "celular"
    re.compile(r'\b[a-z]\d\b', re.IGNORECASE), # contém, p.e., "G5", "S9", ...
    re.compile(r'\bcapa\b', re.IGNORECASE), # contém "capa"
    re.compile(r'\bpara\b', re.IGNORECASE), # contém "para" (p.e. "antena para celular")
]

# Usado para nomear as colunas do DataFrame
attr_names = ['smart', 'phone', 'celular', 'letra_num', 'capa', 'para']

In [81]:
# Transforma um título em uma lista de atributos
def get_attributes(title):
    title_attributes = []
    for pattern in patterns_re:
        if pattern.search(title) is None:
            title_attributes.append(0)
        else:
            title_attributes.append(1)
    return title_attributes

## Transformação dos dados

In [82]:
attributes = []
for i in annotated_data.index:
    row_attr = get_attributes(annotated_data.loc[i].TITLE)
    attributes.append(row_attr)
annotated_attributes = pd.DataFrame(attributes)
annotated_attributes.columns = attr_names
annotated_attributes.insert(0, 'TITLE', annotated_data.TITLE)
annotated_attributes['smartphone'] = annotated_data.smartphone

annotated_attributes.sample(10).sort_index()

,TITLE,smart,phone,celular,letra_num,capa,para,smartphone
6,Tinta Para Hp Universal High Definition Aton B...,0,0,0,0,0,1,0
69,Capa Lg L4 Ii Tpu Amarelo,0,0,0,1,1,0,0
165,Fonte Carregador Sony Vaio Vgn-Fe590 Vgn-Fe590...,0,0,0,0,0,0,0
239,Smartphone Samsung Galaxy E5 4G Duos E500M/DS ...,1,1,0,1,0,0,1
253,Joico Moisture Recovery Shampoo 300ml,0,0,0,0,0,0,0
344,Capa Samsung Galaxy Win Duos Ultra Slim Vermelha,0,0,0,0,1,0,0
428,Fonte Carregador Sony Vaio Pcg-R600hfp Pcg-R60...,0,0,0,0,0,0,0
460,Omnitrix Touch Omniverse Nova Versão - Sunny 654,0,0,0,0,0,0,0
469,Aplique Madeira e Papel Placa Tartaruga LMAPC-...,0,0,0,0,0,0,0
495,Suporte Veicular 360ª Para Iphone 5/5s / Película,0,1,0,0,0,1,0


Aqui seria cabível uma análise de componentes principais ou análise de correspondência, mas por questões de tempo e simplicidade, deixei todas as colunas como estão.

# Avaliação de classificadores

In [83]:
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

from scipy.stats import ttest_ind

Foi usada a área sob a curva ROC como medida de avaliação.  
Nesse caso, pode ser interessante ajustar as taxas de falsos e verdadeiros positivos, 
logo a curva ROC é uma representação apropriada para a performance de um classificador.

Os classificadores testados foram o perceptron e a máquina de vetores de suporte com kernel polinomial de grau 3.  
Os resultados foram comparados com um teste T pareado para amostras independentes.

In [84]:
perceptron = Perceptron(max_iter=1e3)
svc = SVC(gamma='auto', kernel='poly', degree=3)

def eval_model(X, y):
    roc_auc_per = cross_val_score(
        perceptron,
        X,
        y,
        scoring='roc_auc',
        cv=10
    )
    print('Perceptron:')
    print(f'roc_auc: {np.mean(roc_auc_per):.2f} +- {np.std(roc_auc_per):.2f}\t'
         + f'min: {np.min(roc_auc_per):.2f}; max: {np.max(roc_auc_per):.2f}')
    
    roc_auc_svc = cross_val_score(
        svc,
        X,
        y,
        scoring='roc_auc',
        cv=10
    )
    print('SVC (poly-3):')
    print(f'roc_auc: {np.mean(roc_auc_svc):.2f} +- {np.std(roc_auc_svc):.2f}\t'
         + f'min: {np.min(roc_auc_svc):.2f}; max: {np.max(roc_auc_svc):.2f}')
    
    pvalue = ttest_ind(roc_auc_per, roc_auc_svc).pvalue
    print('=====')
    print(f'p-value: {pvalue:.3f} (5% de significância)')
    if pvalue < 0.05:
        print('Classificadores possuem performances diferentes')
    else:
        print('Não há evidência de diferença de performance')

Foram feitos vários testes, com combinações diferentes das variáveis preditoras.

In [85]:
# Todas as variáveis preditoras
eval_model(
    annotated_attributes.iloc[:, 1:7].values,
    annotated_attributes.iloc[:, 7].values
)

Perceptron:
roc_auc: 0.96 +- 0.03	min: 0.89; max: 0.99
SVC (poly-3):
roc_auc: 0.97 +- 0.02	min: 0.93; max: 0.99
=====
p-value: 0.625 (5% de significância)
Não há evidência de diferença de performance


In [86]:
# Apenas "smart" e "phone"
eval_model(
    annotated_attributes.iloc[:, [1, 2]].values,
    annotated_attributes.iloc[:, 7].values
)

Perceptron:
roc_auc: 0.80 +- 0.06	min: 0.69; max: 0.87
SVC (poly-3):
roc_auc: 0.80 +- 0.06	min: 0.69; max: 0.87
=====
p-value: 0.991 (5% de significância)
Não há evidência de diferença de performance


In [87]:
# Todas exceto "capa" e "para"
eval_model(
    annotated_attributes.iloc[:, 1:5].values,
    annotated_attributes.iloc[:, 7].values
)

Perceptron:
roc_auc: 0.92 +- 0.04	min: 0.84; max: 0.98
SVC (poly-3):
roc_auc: 0.89 +- 0.06	min: 0.76; max: 0.97
=====
p-value: 0.214 (5% de significância)
Não há evidência de diferença de performance


In [88]:
# Todas exceto "celular" e "letra_num"
eval_model(
    annotated_attributes.iloc[:, [1, 2, 5, 6]].values,
    annotated_attributes.iloc[:, 7].values
)

Perceptron:
roc_auc: 0.89 +- 0.02	min: 0.87; max: 0.95
SVC (poly-3):
roc_auc: 0.89 +- 0.02	min: 0.86; max: 0.94
=====
p-value: 0.808 (5% de significância)
Não há evidência de diferença de performance


In [89]:
# Todas exceto "smart" e "phone"
eval_model(
    annotated_attributes.iloc[:, 3:7].values,
    annotated_attributes.iloc[:, 7].values
)

Perceptron:
roc_auc: 0.86 +- 0.05	min: 0.78; max: 0.95
SVC (poly-3):
roc_auc: 0.84 +- 0.05	min: 0.76; max: 0.93
=====
p-value: 0.577 (5% de significância)
Não há evidência de diferença de performance


## Seleção do classificador

Em nenhum dos testes houve evidências para dizer que um dos classificadores teve performance melhor.  
Logo, optei pela simplicidade do perceptron em frente ao SVC. Entre os motivos, há a menor chance de overfitting e menor tempo de treino.

# Transformação do dataset

In [90]:
# TODO